# Data refinement development

In [108]:
import pandas as pd
import matplotlib as plt
import re

In [152]:
class DataLoader:
    def __init__(self, filepath) -> None:
        try:
            self.df = pd.read_csv(filepath)
        except Exception as e:
            print('File could not be read')
            print(f'Error: {e}')
        
    # print error rows and drop from df
    def _error_handler(self, col, errors):
        print(f'The following records contain invalid data types in column \'{col}\': ')
        print(self.df.loc[errors.index, 'Record_Number'].to_string(index = False))
        print('Dropping records from data')
        self.df.drop(index = errors.index, inplace = True)

    # method for checking columns expected to be integeres, with or without 'X' as well
    def _integer_checker(self, col, min = None, max = None, include_x = False):
        data = self.df[col]
        # quicker method if range of acceptable values known
        if max != None and min != None:
            vals = list(range(min, max + 1))
            if include_x == True:
                vals.append('X')
                # if x included, values are stored as strings
                vals = [str(x) for x in vals]
            err = data[~data.isin(vals)]
        # alternative if one of min or max, or all integers
        else:
            err = data[data.apply(lambda x: not str(x).isdigit())]
            data = data.drop(index = err.index).astype(int)
            if max != None:
                err = pd.concat([err, data[data > max]])
            elif min != None:
                err = pd.concat([err, data[data < max]])
            if include_x == True:
                err = err[err != 'X']

        # handle error rows
        if len(err) > 0:
            self._error_handler(col, err)
        else:
            print(f'All data values in column \'{col}\' match expected data type')

    # method to check if values in defined list of possible values
    def _known_values_checker(self, col, vals):
        data = self.df[col]
        err = data[~data.isin(vals)]
        # handle error rows
        if len(err) > 0:
            self._error_handler(col, err)
        else:
            print(f'All data values in column \'{col}\' match expected data type')


    def _string_checker(self, col, regex):
        data = self.df[col]
        err = data[~data.apply(lambda x: bool(regex.match(x)))]
        # handle error rows
        if len(err) > 0:
            self._error_handler(col, err)
        else:
            print(f'All data values in column \'{col}\' match expected data type')

    def refine_data(self):
        self._integer_checker('Record_Number', min = 1)
        self._string_checker('Region', re.compile(r'^[A-Za-z]\d{8}$'))
        self._known_values_checker('RESIDENCE_TYPE', ['P', 'C'])
        self._integer_checker('Family_Composition', min = 0, max = 5, include_x = True)
        self._integer_checker('sex', min = 1, max = 2)
        self._integer_checker('age', min = 1, max = 8)
        self._integer_checker('Marital_Status', min = 1, max = 5)
        self._integer_checker('student', min = 1, max = 2)
        self._integer_checker('Country_Of_Birth', min = 1, max = 2)
        self._integer_checker('health', min = 1, max = 5)
        self._integer_checker('Ethnic_Group', min = 1, max = 6)
        self._integer_checker('religion', min = 1, max = 9)
        self._integer_checker('Economic_Activity', min = 1, max = 9, include_x = True)
        self._integer_checker('Occupation', min = 1, max = 9, include_x = True)
        self._integer_checker('industry', min = 1, max = 13, include_x = True)
        self._integer_checker('Hours_Worked_Per_Week', min = 1, max = 4, include_x = True)
        self._integer_checker('Approximate_Social_Grade', min = 1, max = 4, include_x = True)
    
    def drop_duplicates(self):
        len1 = len(self.df)
        self.df.drop_duplicates(subset = list(self.df.columns).remove('Record_Number'), inplace = True)
        len2 = len(self.df)
        if len1 - len2 != 0:
            print(f'{len1 - len2} duplicated rows removed')
        else:
            print('No duplicated rows found')



    

In [154]:
df_test = DataLoader('D:\\University\\Python for Data Analysis\\Repos\\PFDAAV\\data\\Scotland_teaching_file_1PCT.csv')
df_test.refine_data()
df_test.drop_duplicates()

All data values in column 'Record_Number' match expected data type
All data values in column 'Region' match expected data type
All data values in column 'RESIDENCE_TYPE' match expected data type
All data values in column 'Family_Composition' match expected data type
All data values in column 'sex' match expected data type
All data values in column 'age' match expected data type
All data values in column 'Marital_Status' match expected data type
All data values in column 'student' match expected data type
All data values in column 'Country_Of_Birth' match expected data type
All data values in column 'health' match expected data type
All data values in column 'Ethnic_Group' match expected data type
All data values in column 'religion' match expected data type
All data values in column 'Economic_Activity' match expected data type
All data values in column 'Occupation' match expected data type
All data values in column 'industry' match expected data type
All data values in column 'Hours_Wo

In [4]:
df = pd.read_csv('D:\\University\\Python for Data Analysis\\Repos\\PFDAAV\\data\\Scotland_teaching_file_1PCT.csv')

In [117]:
for col in df.columns:
    print([col, df[col].unique()])

['Record_Number', array([    1,     2,     3, ..., 63386, 63387, 63388], dtype=int64)]
['Region', array(['S92000003'], dtype=object)]
['RESIDENCE_TYPE', array(['P', 'C'], dtype=object)]
['Family_Composition', array(['1', '0', '4', '2', '5', '3', 'X'], dtype=object)]
['sex', array([1, 2], dtype=int64)]
['age', array([4, 3, 6, 5, 1, 2, 7, 8], dtype=int64)]
['Marital_Status', array([2, 1, 4, 3, 5], dtype=int64)]
['student', array([2, 1], dtype=int64)]
['Country_Of_Birth', array([2, 1], dtype=int64)]
['health', array([2, 3, 1, 4, 5], dtype=int64)]
['Ethnic_Group', array([1, 2, 4, 3, 6, 5], dtype=int64)]
['religion', array([5, 1, 2, 9, 6, 8, 4, 7, 3], dtype=int64)]
['Economic_Activity', array(['1', 'X', '5', '6', '9', '4', '2', '7', '3', '8'], dtype=object)]
['Occupation', array(['5', '1', '4', '2', '3', '9', 'X', '8', '6', '7'], dtype=object)]
['industry', array(['5', '8', '11', '9', 'X', '2', '4', '6', '7', '10', '3', '12',
       '13', '1'], dtype=object)]
['Hours_Worked_Per_Week', array

In [118]:
df

,Record_Number,Region,RESIDENCE_TYPE,Family_Composition,sex,age,Marital_Status,student,Country_Of_Birth,health,Ethnic_Group,religion,Economic_Activity,Occupation,industry,Hours_Worked_Per_Week,Approximate_Social_Grade
0,1,S92000003,P,1,1,4,2,2,2,2,1,5,1,5,5,4,3
1,2,S92000003,P,0,1,3,1,2,1,2,1,1,1,1,8,3,2
2,3,S92000003,P,1,2,4,2,2,1,3,1,1,1,4,8,2,4
3,4,S92000003,P,1,2,6,2,2,1,1,1,1,1,2,8,4,2
4,5,S92000003,P,4,2,4,4,2,1,2,1,1,1,3,11,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63383,63384,S92000003,P,1,1,2,1,1,1,1,1,2,6,X,X,X,3
63384,63385,S92000003,P,1,1,4,2,2,1,1,1,1,1,3,9,3,2
63385,63386,S92000003,P,1,1,5,2,2,1,1,1,9,1,8,3,4,4
63386,63387,S92000003,P,4,2,1,1,1,1,1,1,9,X,X,X,X,X
